In [ ]:
# importing libraries

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score
import random

In [ ]:
# Setting the random seeds for the reproducibility of the results
# The segments of the code were taken from
# https://keras.io/api/utils/python_utils/#set_random_seed-function
# https://github.com/keras-team/keras

tf.random.set_seed(5)
np.random.seed(5)
random.seed(5)

In [ ]:
# Setting image size and batch size
img_size = (128, 128)
batch_size = 64

In [ ]:
# Data Augmentation and Normalization for Train, Validation, and Test sets
train_datagen = ImageDataGenerator(rescale=1.0/255)
validate_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)


# Load Train, Validation, and Test data from directories
train_data = train_datagen.flow_from_directory(
    'dataset/aug2/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'  
)

val_data = validate_datagen.flow_from_directory(
    'dataset/aug2/val',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    'dataset/aug2/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  
)

In [ ]:
# Loading the EfficientNetV2L model with ImageNet weights, excluding top layers
base_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze only the last 5 layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Training the model

# Train the model on your dataset with class weights and early stopping
history = model.fit(train_data, epochs=60, validation_data=val_data)

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# Calculate Precision, Recall, and F1-Score
test_data.reset()
Y_pred = model.predict(test_data)
y_pred = (Y_pred > 0.5).astype(int)
y_true = test_data.classes

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

In [ ]:
# Confusion Matrix

# Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_data.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.savefig('confusion_matrix2.png')
plt.show()

In [ ]:
# Plot Training & Validation Accuracy

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
plt.xlim(0, 33)
plt.ylim(0.5, 0.9)
plt.savefig('model_accuracy.png')
plt.show()

In [ ]:
# Plot Training & Validation Loss

plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.xlim(0, 33)
plt.ylim(0.3, 0.8)
plt.savefig('model_loss.png')
plt.show()

In [ ]:
# Compute ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, Y_pred)
auc_score = roc_auc_score(y_true, Y_pred)
print(f"AUC Score: {auc_score:.2f}")

In [ ]:
# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid()
plt.savefig('roc_auc_curve.png')
plt.show()

In [ ]:
np.save('y_true_model2.npy', y_true)
np.save('y_pred_proba_model2.npy', y_pred)